In [3]:
from vnstock import *
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
data_ticker = pd.read_excel("C:\\Users\\Chi Bao\\OneDrive\\Desktop\\Đồ án tốt nghiệp\\data_ticker.xlsx")

FI = ['GC=F','SI=F','CL=F','^GSPC','^DJI','^IXIC','^RUT','^FTSE','^N225','^NYA']
df = pd.DataFrame()
for symbol in FI:
    data = yf.download(symbol, start='2012-03-20', end='2023-09-06')
    df[symbol] = data['Adj Close']

column_mapping = {
    'GC=F': 'Gold',
    'SI=F': 'Silver',
    'CL=F': 'Crude Oil',
    '^GSPC': 'S&P 500',
    '^DJI': 'Dow Jones',
    '^IXIC': 'NASDAQ',
    '^RUT': 'Russell 2000',
    '^FTSE': 'FTSE 100',
    '^N225': 'Nikkei 225',
    '^NYA': 'NYSE'
}

df = df.rename(columns=column_mapping)
df.dropna()

returns_df = df.pct_change()*100
returns_df.columns = [col + '_return' for col in returns_df.columns]
returns_df = returns_df.dropna()

###

FI = ['VNINDEX','HNX','VN30','HNX30','UPCOM']
data = stock_historical_data('VNINDEX', '2012-03-19', '2023-09-05', "1D", "index")
d_new = data.set_index('time')
d_new['VNINDEX'] = data['close']
df1 = pd.DataFrame()
for symbol in FI:
    data = stock_historical_data(symbol, '2012-03-19', '2023-09-05', "1D", "index")
    data = data.set_index('time')
    df1[symbol] = data['close']
df1 = df1.dropna()

returns_df1 = df1.pct_change()*100
returns_df1.columns = [col + '_return' for col in returns_df1.columns]
returns_df1 = returns_df1.dropna()
returns_df1.head(42069)

###

lagreturn = pd.DataFrame()

for column in returns_df.columns:
    lagreturn[f'{column}_Lag1'] = returns_df[column].shift(1)
for column in returns_df.columns:
    lagreturn[f'{column}_Lag2'] = returns_df[column].shift(2)
lagreturn = lagreturn.dropna()

lagreturn1 = pd.DataFrame()
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag1'] = returns_df1[column].shift(1)
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag2'] = returns_df1[column].shift(2)
lagreturn1 = lagreturn1.dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
def do_tre_vnstock(symbol, start_date, end_date, lags=10): 
    ts = stock_historical_data(symbol,start_date, end_date)
    ts = ts.dropna(subset=['close'])


    tsret = pd.DataFrame(index=ts.index)
    tsret['time']=ts['time']
    tsret["Today"] = ts["close"].pct_change()*100

    for i,x in enumerate(tsret["Today"]):
        if (abs(x) < 0.0001):
            tsret["Today"][i] = 0.0001

    for i in range(0, lags):
        tsret["Lag%s" % str(i+1)] = tsret["Today"].shift(i+1)

    tsret["Direction"] = np.sign(tsret["Today"])
    tsret = tsret.dropna()
    tsret = tsret.set_index('time')
    bpt = pd.merge(tsret, lagreturn, left_index=True, right_index=True)
    bpt1 = pd.merge(bpt, lagreturn1, left_index=True, right_index=True)
    bpt1.fillna(0, inplace=True)
    bpt1 = bpt1[(bpt1 != 0).all(axis=1)]
    return bpt1

In [5]:
#MODEL
list_i = []
list_return_price = []
list_rmse = []
list_mae =[]
list_mse = []
list_r2_train = []
list_r2_test = []

for i in data_ticker['ticker']:
  try:
    data = do_tre_vnstock(i, '2012-03-19', '2023-09-05', lags=10)

    X = data.loc[:, ~data.columns.isin(['Today'])]
    y = data["Today"]

    model = Ridge(alpha=1.0)
    X_train, X_test,y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=12, shuffle=False)
    model.fit(X_train,y_train)
    yhat_test = model.predict(X_test)
    r2_train = model.score(X_train, y_train)
    r2_test = model.score(X_test, y_test)
    mse = mean_squared_error(y_test, yhat_test, squared=True)
    rmse = mean_squared_error(y_test, yhat_test, squared=False)
    mae = mean_absolute_error(y_test, yhat_test)

    X_predict = [X.iloc[-1].values.tolist()]
    Y_predict = model.predict(X_predict)

    list_i.append(i)
    list_return_price.append(Y_predict)
    list_r2_train.append(r2_train)
    list_r2_test.append(r2_test)
    list_rmse.append(rmse)
    list_mse.append(mse)
    list_mae.append(mae)

    print('Stock:',i,f' Predict: {Y_predict.item():.2f}',f' R2_train: {r2_train:.2f}',f' R2_test: {r2_test:.2f}',
          f' RMSE: {mse:.2f}',f' RMSE: {mse:.2f}', f'MAE: {mae:.2f}')
  except Exception as e:
    print(f"Stock: {i} -Unpredictable!\n" f'{e}')
    continue

Stock: SSI  Predict: -2.63  R2_train: 0.59  R2_test: 0.45  RMSE: 2.56  RMSE: 2.56 MAE: 1.36
Stock: BCM  Predict: -1.24  R2_train: 0.46  R2_test: -0.48  RMSE: 1.73  RMSE: 1.73 MAE: 1.12
Stock: VHM  Predict: 1.75  R2_train: 0.51  R2_test: 0.52  RMSE: 2.38  RMSE: 2.38 MAE: 1.12
Stock: VIC  Predict: 1.25  R2_train: 0.50  R2_test: 0.31  RMSE: 2.89  RMSE: 2.89 MAE: 1.18
Stock: VRE  Predict: -1.66  R2_train: 0.56  R2_test: 0.46  RMSE: 1.74  RMSE: 1.74 MAE: 1.08
Stock: BVH  Predict: 1.41  R2_train: 0.58  R2_test: 0.18  RMSE: 1.14  RMSE: 1.14 MAE: 0.92
Stock: POW  Predict: 1.71  R2_train: 0.58  R2_test: 0.38  RMSE: 1.71  RMSE: 1.71 MAE: 1.07
Stock: GAS  Predict: 1.49  R2_train: 0.56  R2_test: 0.03  RMSE: 1.11  RMSE: 1.11 MAE: 0.92
Stock: ACB  Predict: 1.30  R2_train: 0.53  R2_test: 0.27  RMSE: 1.20  RMSE: 1.20 MAE: 0.95
Stock: BID  Predict: 1.57  R2_train: 0.55  R2_test: 0.42  RMSE: 1.61  RMSE: 1.61 MAE: 1.02
Stock: CTG  Predict: 1.75  R2_train: 0.57  R2_test: 0.39  RMSE: 1.51  RMSE: 1.51 MAE: 

In [6]:
KQ = pd.DataFrame({'Stock': list_i, 'Predict': list_return_price, 'R2_train': list_r2_train,'R2_test': list_r2_test, 'RMSE': list_mse,'MSE': list_rmse, 'MAE': list_mae,})
KQ.to_excel("Kết quả 20% test size.xlsx")
KQ.head(42069)

,Stock,Predict,R2_train,R2_test,RMSE,MSE,MAE
0,SSI,[-2.631908803593841],0.594267,0.454956,2.560657,1.600205,1.357682
1,BCM,[-1.2430358388047826],0.459690,-0.475871,1.732112,1.316097,1.118402
2,VHM,[1.7459180694382972],0.514295,0.520513,2.376366,1.541547,1.120249
3,VIC,[1.2514084913659955],0.497883,0.312198,2.890157,1.700046,1.180295
4,VRE,[-1.659082403986465],0.555930,0.457924,1.741659,1.319719,1.083937
...,...,...,...,...,...,...,...
389,TDP,[0.2168310792092451],0.363407,0.247976,4.469007,2.114003,1.828693
390,NO1,[-1.5159378308492075],0.601097,0.330084,7.825200,2.797356,2.171802
391,L10,[1.5356129810972126],0.649031,0.574833,15.404829,3.924899,2.939975
392,REE,[1.4122957276673136],0.517384,0.266742,1.333034,1.154571,1.005609
